# 검증 데이터 셋으로 테스트


# 1. 환경 세팅

In [97]:
%load_ext autoreload
%autoreload 2

import sys, os

local_module_path = "../"
sys.path.append(os.path.abspath(local_module_path))
print("local_module_path: ",os.path.abspath(local_module_path))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
local_module_path:  /home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/03_reranker_hybrid_search


## Bedrock Client 생성
### 참고
- 아래의 노트북은 베드락의 사용을 초기화 합니다. 추후에 베드락의 임베딩 모델등을 사용하기 위함 입니다.



In [98]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from local_utils import bedrock, print_ww
from local_utils.bedrock import bedrock_info

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models())


Create new client
  Using region: us-east-1
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)

== FM lists ==
############### bedrock info
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Claude-V2-1': 'anthropic.claude-v2:1',
 'Cohere-Embeddings-En': 'cohere.embed-english-v3',
 'Cohere-Embeddings-Multilingual': 'cohere.embed-multilingual-v3',
 'Command': 'cohere.command-text-v14',
 'Command-Light': 'cohere.command-light-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Llama2-13b-Chat': 'meta.llama2-13b-chat-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'amazon.titan-text-express-v1',
 'Titan-Text-G1-Light': 'amazon.titan-text-lite-v1'}


## Embedding 모델 선택

In [99]:
from local_utils.rag import (
    KoSimCSERobertaContentHandler, 
    SagemakerEndpointEmbeddingsJumpStart,
    get_embedding_model
)

#### [중요] is_KoSimCSERobert == True 경우,  endpoint_name 을 꼭 넣어 주세요.

In [100]:
 %store -r koSimCSE_endpoint_name
print("kosimcse_endpoint_name: \n", koSimCSE_endpoint_name)

kosimcse_endpoint_name: 
 KoSimCSE-roberta-2024-02-12-06-54-55


In [101]:
is_bedrock_embeddings = True
is_KoSimCSERobert = False

aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

##############################
# Parameters for is_KoSimCSERobert
##############################
if is_KoSimCSERobert: endpont_name = koSimCSE_endpoint_name
else: endpont_name = None
##############################

llm_emb = get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, boto3_bedrock, aws_region, endpont_name)    

####################
model_name:  Titan-Embeddings-G1
Bedrock Embeddings Model Loaded


# 2. OpenSearch Client 생성
    
#### [중요] 아래에 aws parameter store 에 아래 인증정보가 먼저 입력되어 있어야 합니다.

In [102]:
from local_utils.proc_docs import get_parameter

In [103]:
aws_region = 'us-east-1'
ssm = boto3.client("ssm", aws_region)

opensearch_domain_endpoint = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'lec_opensearch_domain_endpoint',
)

opensearch_user_id = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'lec_opensearch_userid',
)

opensearch_user_password = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'lec_opensearch_password',
)

http_auth = (opensearch_user_id, opensearch_user_password) # Master username, Master password

In [104]:
from local_utils.opensearch import opensearch_utils

In [105]:
os_client = opensearch_utils.create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)

# 3. OpenSearch 벡터 Index 확인


## 오픈 서치 인덱스 확인 


In [106]:
%store -r index_name

In [107]:
# index_name = "lecm-intent-v3"
%store -r index_name
print("index_name: ", index_name)

index_name:  search_retail_demo_store


In [108]:

index_exists = opensearch_utils.check_if_index_exists(os_client, index_name)

if index_exists:
    print("Index is ready")



index_name=search_retail_demo_store, exists=True
Index is ready


## 랭체인 인덱스 연결 오브젝트 생성

- [langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html)

In [109]:
from langchain.vectorstores import OpenSearchVectorSearch

vector_db = OpenSearchVectorSearch(
    index_name=index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=llm_emb,
    http_auth=http_auth, # http_auth
    is_aoss =False,
    engine="faiss",
    space_type="l2",
    bulk_size=100000,
    timeout=60    
)
vector_db

# 4. 검증 데이터 읽기
- /data/aws_opensearch_data_validate.csv

In [110]:
import pandas as pd 
import os

dataset_folder = "../data/retail_demo_store/"

data_file_path =  os.path.join(dataset_folder,"input_raw/test_kr_search_retail_demo.csv")
# data_file_path =  os.path.join(dataset_folder,"input_raw/aws_opensearch_data_validate.csv")

dfv =  pd.read_csv(data_file_path)
dfv['input']

0                       남성용 블랙 포멀 슈즈 한 켤레
1                     모든 욕실에 사용할 수 있는 면도기
2        우리 재배자들이 신선하고 활기차게 배달한 백합 어레인지먼트
3       이 어쿠스틱 드럼은 아마추어 및 전문 뮤지션에게 적합합니다.
4      이 어쿠스틱 베이스는 아마추어 및 전문 음악가에게 적합합니다.
                      ...                
370      우리 돼지고기는 농부들이 지속 가능한 방식으로 사육합니다.
371            어떤 상황에도 어울리는 여성용 울트라시크 스카프
372              모든 상황에 어울리는 고급스러운 세라믹 그릇
373         이 새들 브라운 드레시 핸드백은 전형적인 필수품입니다
374    이 스틸 블루 스퀘어 쿠션은 집에 꼭 있어야 할 필수품입니다.
Name: input, Length: 375, dtype: object

In [111]:
import os

output_folder_name = os.path.join(dataset_folder, "output" )
os.makedirs(output_folder_name, exist_ok=True)

# 5. Lexical Search

In [139]:
from local_utils.rag import (evaluate_search_methods)
# def evaluate_search_methods(output_file_path, search_method, eval_df, verbose=False):

output_file_path = os.path.join(dataset_folder, "output/aws_opensearch_result01.csv" )    
evaluate_search_methods(output_file_path=output_file_path, 
                        parallel = True,
                        num_process = 30, # how many processes to run
                        num_eval = 1200, # how many samples to run, now max is 1200
                        search_method='lexical', 
                        eval_df = dfv, 
                        vector_db = vector_db,
                        single_num_result = 3,
                        hybrid_num_result = 1,
                        index_name = index_name,
                        os_client = os_client,
                        minimum_should_match = 50,
                        ensemble_weights = [0.50, 0.50],
                        verbose= False )

##############################
parallel way:  True
search method:  lexical
##############################
../data/retail_demo_store/output/aws_opensearch_result01.csv is saved


In [113]:

pd.set_option('display.max_rows', 100)
df = pd.read_csv(output_file_path)
df

,Unnamed: 0,product,input,intent,result
0,0,블랙 슈즈,남성용 블랙 포멀 슈즈 한 켤레,신발|정장,신발|정장
1,1,스틸 가위,모든 욕실에 사용할 수 있는 면도기,뷰티|그루밍,뷰티|그루밍
2,2,백합 어레인지먼트,우리 재배자들이 신선하고 활기차게 배달한 백합 어레인지먼트,플로럴|어레인지먼트,플로럴|어레인지먼트
3,3,어쿠스틱 드럼,이 어쿠스틱 드럼은 아마추어 및 전문 뮤지션에게 적합합니다.,악기|타악기,악기|타악기
4,4,어쿠스틱 베이스,이 어쿠스틱 베이스는 아마추어 및 전문 음악가에게 적합합니다.,악기 | 현악기,악기 | 현악기
...,...,...,...,...,...
370,370,돼지고기,우리 돼지고기는 농부들이 지속 가능한 방식으로 사육합니다.,식료품|고기,식료품|고기
371,371,울트라시크 스카프,어떤 상황에도 어울리는 여성용 울트라시크 스카프,의류|스카프,의류|스카프
372,372,고급스러운 세라믹 볼,모든 상황에 어울리는 고급스러운 세라믹 그릇,가정용품|그릇,가정용품|그릇
373,373,새들 브라운 핸드백,이 새들 브라운 드레시 핸드백은 전형적인 필수품입니다,액세서리|핸드백,액세서리|핸드백


In [114]:
df.result.value_counts()

result
가정용품|주방       33
계절|크리스마스      19
가구|테이블        19
가구|의자         16
의류|재킷         16
신발|정장         16
의류|셔츠         15
가정 장식 | 장식    13
악기 | 현악기      12
가정용품|그릇       12
식료품|베이커리      11
액세서리|안경       11
홈 데코 | 조명     11
액세서리|배낭       10
가구|소파         10
액세서리|벨트        9
신발|스니커즈        9
보석|팔찌          9
홈 데코 | 쿠션      9
계절별|할로윈        9
의류|스카프         8
액세서리|핸드백       8
계절|부활절         8
식료품|야채         8
꽃|식물           8
악기|타악기         8
플로럴|어레인지먼트     7
계절별|발렌타인       7
야외|낚시          7
악기|키           7
신발|샌들          7
전자제품|카메라       7
식료품|고기         6
가구|옷장          6
뷰티|그루밍         4
Name: count, dtype: int64

In [115]:
df[df.result == "No Result"]

,Unnamed: 0,product,input,intent,result


# 6. Semantic Search
- query 를 제공해서 실제로 유사한 내용이 검색이 되는지를 확인 합니다.

In [138]:
output_file_path = os.path.join(dataset_folder, "output/aws_opensearch_result02.csv" )    
evaluate_search_methods(output_file_path=output_file_path, 
                        parallel = True,
                        num_process = 30,
                        num_eval = 1200, # how many samples to run                        
                        search_method='semantic', 
                        eval_df = dfv, 
                        vector_db = vector_db,
                        single_num_result = 3,
                        hybrid_num_result = 1,
                        index_name = index_name,
                        os_client = os_client,
                        minimum_should_match = 50,
                        ensemble_weights = [0.50, 0.50],
                        verbose= False )


##############################
parallel way:  True
search method:  semantic
##############################
../data/retail_demo_store/output/aws_opensearch_result02.csv is saved


In [117]:
df = pd.read_csv(output_file_path)
df

,Unnamed: 0,product,input,intent,result
0,0,블랙 슈즈,남성용 블랙 포멀 슈즈 한 켤레,신발|정장,신발|정장
1,1,스틸 가위,모든 욕실에 사용할 수 있는 면도기,뷰티|그루밍,뷰티|그루밍
2,2,백합 어레인지먼트,우리 재배자들이 신선하고 활기차게 배달한 백합 어레인지먼트,플로럴|어레인지먼트,플로럴|어레인지먼트
3,3,어쿠스틱 드럼,이 어쿠스틱 드럼은 아마추어 및 전문 뮤지션에게 적합합니다.,악기|타악기,악기|타악기
4,4,어쿠스틱 베이스,이 어쿠스틱 베이스는 아마추어 및 전문 음악가에게 적합합니다.,악기 | 현악기,악기 | 현악기
...,...,...,...,...,...
370,370,돼지고기,우리 돼지고기는 농부들이 지속 가능한 방식으로 사육합니다.,식료품|고기,식료품|고기
371,371,울트라시크 스카프,어떤 상황에도 어울리는 여성용 울트라시크 스카프,의류|스카프,의류|스카프
372,372,고급스러운 세라믹 볼,모든 상황에 어울리는 고급스러운 세라믹 그릇,가정용품|그릇,가정용품|그릇
373,373,새들 브라운 핸드백,이 새들 브라운 드레시 핸드백은 전형적인 필수품입니다,액세서리|핸드백,액세서리|핸드백


In [118]:
df.result.value_counts()

result
가정용품|주방       32
계절|크리스마스      19
가구|의자         18
가구|테이블        17
의류|재킷         16
신발|정장         16
의류|스카프        14
액세서리|안경       13
가정 장식 | 장식    13
가구|소파         12
의류|셔츠         12
악기 | 현악기      12
홈 데코 | 조명     11
가정용품|그릇       11
식료품|베이커리      11
액세서리|배낭       10
액세서리|핸드백       9
신발|스니커즈        9
홈 데코 | 쿠션      9
계절별|할로윈        9
액세서리|벨트        8
식료품|야채         8
계절|부활절         8
악기|타악기         8
꽃|식물           8
악기|키           7
플로럴|어레인지먼트     7
계절별|발렌타인       7
야외|낚시          7
전자제품|카메라       7
신발|샌들          6
가구|옷장          6
식료품|고기         6
보석|팔찌          5
뷰티|그루밍         4
Name: count, dtype: int64

# 7. Hybrid Search

In [137]:
output_file_path = os.path.join(dataset_folder, "output/aws_opensearch_result03.csv" )    
evaluate_search_methods(output_file_path=output_file_path, 
                        parallel = True,
                        num_process = 30,
                        num_eval = 1200, # how many samples to run, now max is 1200                        
                        search_method='hybrid', 
                        eval_df = dfv, 
                        vector_db = vector_db,
                        single_num_result = 10,
                        hybrid_num_result = 1,
                        index_name = index_name,
                        os_client = os_client,
                        minimum_should_match = 50,
                        ensemble_weights = [0.5, 0.5], # the first arg: semantic, second arg : lexical
                        verbose= False )


##############################
parallel way:  True
search method:  hybrid
##############################
../data/retail_demo_store/output/aws_opensearch_result03.csv is saved


In [127]:
df = pd.read_csv(output_file_path)
df

,Unnamed: 0,product,input,intent,result
0,0,블랙 슈즈,남성용 블랙 포멀 슈즈 한 켤레,신발|정장,신발|정장
1,1,스틸 가위,모든 욕실에 사용할 수 있는 면도기,뷰티|그루밍,뷰티|그루밍
2,2,백합 어레인지먼트,우리 재배자들이 신선하고 활기차게 배달한 백합 어레인지먼트,플로럴|어레인지먼트,플로럴|어레인지먼트
3,3,어쿠스틱 드럼,이 어쿠스틱 드럼은 아마추어 및 전문 뮤지션에게 적합합니다.,악기|타악기,악기|타악기
4,4,어쿠스틱 베이스,이 어쿠스틱 베이스는 아마추어 및 전문 음악가에게 적합합니다.,악기 | 현악기,악기 | 현악기
...,...,...,...,...,...
370,370,돼지고기,우리 돼지고기는 농부들이 지속 가능한 방식으로 사육합니다.,식료품|고기,식료품|고기
371,371,울트라시크 스카프,어떤 상황에도 어울리는 여성용 울트라시크 스카프,의류|스카프,의류|스카프
372,372,고급스러운 세라믹 볼,모든 상황에 어울리는 고급스러운 세라믹 그릇,가정용품|그릇,가정용품|그릇
373,373,새들 브라운 핸드백,이 새들 브라운 드레시 핸드백은 전형적인 필수품입니다,액세서리|핸드백,액세서리|핸드백


In [128]:
df.result.value_counts()

result
가정용품|주방       33
계절|크리스마스      19
가구|테이블        19
가구|의자         16
의류|재킷         16
신발|정장         16
가정 장식 | 장식    13
의류|스카프        12
의류|셔츠         12
악기 | 현악기      12
액세서리|안경       12
가정용품|그릇       11
가구|소파         11
식료품|베이커리      11
홈 데코 | 조명     11
액세서리|배낭       10
신발|스니커즈       10
홈 데코 | 쿠션      9
계절별|할로윈        9
액세서리|핸드백       8
액세서리|벨트        8
악기|타악기         8
계절|부활절         8
식료품|야채         8
꽃|식물           8
보석|팔찌          8
플로럴|어레인지먼트     7
계절별|발렌타인       7
야외|낚시          7
악기|키           7
전자제품|카메라       7
신발|샌들          6
식료품|고기         6
가구|옷장          6
뷰티|그루밍         4
Name: count, dtype: int64

# 8. Hybrid + Reranker

In [129]:
%store -r reranker_endpoint_name
print("reranker_endpoint_name: ", reranker_endpoint_name)


reranker_endpoint_name:  Ko-Reranker-2024-02-12-06-58-12


In [135]:
output_file_path = os.path.join(dataset_folder, "output/aws_opensearch_result04.csv" )
evaluate_search_methods(output_file_path=output_file_path, 
                        parallel = True, # False: single process, True: parallel ways
                        num_process = 30,
                        num_eval = 1200, # how many samples to run, now max is 1200                        
                        search_method='reranker', 
                        eval_df = dfv, 
                        vector_db = vector_db,
                        single_num_result = 5,
                        hybrid_num_result = 5,
                        index_name = index_name,
                        os_client = os_client,
                        minimum_should_match = 10,
                        ensemble_weights = [0.50, 0.50], # the first argument is semantic and the second one is lexical
                        reranker_endpoint_name = reranker_endpoint_name,
                        verbose= False )




##############################
parallel way:  True
search method:  reranker
##############################
../data/retail_demo_store/output/aws_opensearch_result04.csv is saved


In [133]:
df = pd.read_csv(output_file_path)
df

,Unnamed: 0,product,input,intent,result
0,0,블랙 슈즈,남성용 블랙 포멀 슈즈 한 켤레,신발|정장,신발|정장
1,1,스틸 가위,모든 욕실에 사용할 수 있는 면도기,뷰티|그루밍,뷰티|그루밍
2,2,백합 어레인지먼트,우리 재배자들이 신선하고 활기차게 배달한 백합 어레인지먼트,플로럴|어레인지먼트,플로럴|어레인지먼트
3,3,어쿠스틱 드럼,이 어쿠스틱 드럼은 아마추어 및 전문 뮤지션에게 적합합니다.,악기|타악기,악기|타악기
4,4,어쿠스틱 베이스,이 어쿠스틱 베이스는 아마추어 및 전문 음악가에게 적합합니다.,악기 | 현악기,악기 | 현악기
...,...,...,...,...,...
370,370,돼지고기,우리 돼지고기는 농부들이 지속 가능한 방식으로 사육합니다.,식료품|고기,식료품|고기
371,371,울트라시크 스카프,어떤 상황에도 어울리는 여성용 울트라시크 스카프,의류|스카프,의류|스카프
372,372,고급스러운 세라믹 볼,모든 상황에 어울리는 고급스러운 세라믹 그릇,가정용품|그릇,가정용품|그릇
373,373,새들 브라운 핸드백,이 새들 브라운 드레시 핸드백은 전형적인 필수품입니다,액세서리|핸드백,액세서리|핸드백


In [134]:
df.result.value_counts()

result
가정용품|주방       32
계절|크리스마스      19
가구|테이블        18
가구|의자         17
의류|재킷         16
신발|정장         16
액세서리|안경       14
의류|셔츠         13
가정 장식 | 장식    13
악기 | 현악기      12
의류|스카프        11
가정용품|그릇       11
홈 데코 | 조명     11
식료품|베이커리      11
가구|소파         10
홈 데코 | 쿠션     10
액세서리|배낭       10
신발|스니커즈        9
계절별|할로윈        9
계절별|발렌타인       8
액세서리|벨트        8
악기|타악기         8
계절|부활절         8
식료품|야채         8
꽃|식물           8
악기|키           7
플로럴|어레인지먼트     7
액세서리|핸드백       7
야외|낚시          7
전자제품|카메라       7
뷰티|그루밍         6
보석|팔찌          6
식료품|고기         6
신발|샌들          6
가구|옷장          6
Name: count, dtype: int64